In [1]:
# TODO address all TODOs in outage_practice_helper_functions.ipynb
%run outage_practice_helper_functions.ipynb
%run AMINonVee.ipynb

In [ ]:
# def build_aggregate_of_meters_df():
#     # Previously was build_df_aggregated_for_each_time_index


# def build_time_resampled_df():
#     # Previously was build_resampled_df


# def build_time_resampled_aggregate_of_meters_df():
#     # Previously was build_resampled_df_aggregated_for_each_time_index
#     # First, perform aggregation of meters
#     # Then, time resample


# def build_time_resampled_dfs
#     # Previously get_resampled_dfs

# outage_practice_helper_functions exists here ONLY to double check and compare old methods with new

In [2]:
from importlib import reload
#reload(Utilities)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version
import itertools
from dateutil.parser import parse
from operator import itemgetter

from pmdarima import auto_arima
import statsmodels.api as sm
from statsmodels.tsa.stattools import acovf, acf, pacf, pacf_yw, pacf_ols
from pandas.plotting import lag_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics.tsaplots import month_plot, quarter_plot, seasonal_plot
from statsmodels.tsa.arima_model import ARMA, ARIMA, ARMAResults, ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX

from arch import arch_model

from scipy.stats.mstats import trim

#---------------------------------------------------------------------
import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
# import constants for the days of the week
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Plot_Box_sns
import GrubbsTest
import DickeyFullerTest as dft

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def build_mean_from_sum_and_counts(df, sum_x_col, n_counts_col, placement_col):
    df[placement_col] = df[sum_x_col]/df[n_counts_col]
    return df

def build_std_from_mossom(df, sum_x_col, sum_x2_col, n_counts_col, placement_col, sample_std=True):
    # mossom = Mean Of Sqaures minus Square Of Means
    #   variance = std**2 ~ bar{x**2}-(bar{x})**2
    # However, here I will actually be using sum(x**2) and sum(x) instead
    #   ==> std**2 ~ (1/n)*sum(x**2) - (1/n**2)*sum(x)
    # Default to sample std (std_s w/ n-1 in denominator), 
    #    not population std (std_p w/ n in denominator)
    # 
    # std_p**2 = bar{x**2}-(bar{x})**2
    #          = (1/n)*sum(x**2) - (1/n**2)*sum(x)
    # std_s**2 = (n/(n-1))*std_p**2 = (n/(n-1))*(bar{x**2}-(bar{x})**2)
    #          = (n/(n-1))*((1/n)*sum(x**2) - (1/n**2)*sum(x))
    df[placement_col] = np.sqrt(
        (df[sum_x2_col]/df[n_counts_col] - df[sum_x_col]*df[sum_x_col]/(df[n_counts_col]*df[n_counts_col]))
    )
    if sample_std:
        df[placement_col] = df[placement_col]*np.sqrt((df[n_counts_col]/(df[n_counts_col]-1)))
    return df

def build_mean_and_std_from_sum_x_x2_and_counts(df, sum_x_col, sum_x2_col, n_counts_col, 
                                                placement_col_mean, placement_col_std, 
                                                sample_std=True):
    df = build_mean_from_sum_and_counts(df, sum_x_col, n_counts_col, placement_col_mean)
    df = build_std_from_mossom(df, sum_x_col, sum_x2_col, n_counts_col, placement_col_std)
    return df

In [5]:
cols_of_interest = [
    'serialnumber',
    'prem_nb', 
    'srvc_pole_nb', 
    'trsf_pole_nb',
    'annual_kwh',
    'station_nb',
    'xfmr_nb', 
    'starttimeperiod', 
    'endtimeperiod', 
    'aep_endtime_utc', 
    'timezoneoffset', 
    'aep_derived_uom',
    'aep_srvc_qlty_idntfr', 
    'value', 
    'aep_usage_dt'
]

cols_of_interest_cir = [
    'starttimeperiod', 
    'endtimeperiod', 
    'aep_endtime_utc', 
    'timezoneoffset',
    'aep_derived_uom', 
    'aep_srvc_qlty_idntfr', 
    'aep_usage_dt', 
    'value_sum',
    'counts', 
    'value_mean'
]

In [6]:
df_outage_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_outages\outg_rec_nb_11751094')
#df_outage_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_outages_full\outg_rec_nb_11751094')
df_circuit_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_circuits\outg_rec_nb_11751094')
#df_circuit_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_circuits\Batch2_LessCounts\outg_rec_nb_11751094')

out_t_beg_loc = pd.to_datetime('2020-06-21 10:52:00')
out_t_end_loc = pd.to_datetime('2020-06-21 12:50:00')
no_outage_center_loc = pd.to_datetime('2020-08-15')

In [7]:
freqs=['H', '4H', 'D', 'MS']
time_col_for_agg='endtimeperiod_utc'
other_grouper_cols=['serialnumber']
build_agg_dfs=True
other_cols_to_keep = ['prem_nb', 'srvc_pole_nb', 'trsf_pole_nb', 'station_nb', 
                      'xfmr_nb', 'aep_derived_uom', 'aep_srvc_qlty_idntfr']
other_cols_to_keep_agg = ['aep_derived_uom', 'aep_srvc_qlty_idntfr']
agg_cols=['value', 'annual_kwh']
agg_types=['mean']
mix_agg_functions=False

# Should not be any overlap between other_cols_to_keep and agg_cols
assert(len(set(other_cols_to_keep).intersection(set(agg_cols)))==0)
assert(len(set(other_cols_to_keep_agg).intersection(set(agg_cols)))==0)

In [8]:
df_key = 'df'
df_agg_key = 'df_agg'

In [1]:
kwh_vlt_dfs_dict = assemble_kwh_vlt_dfs_from_saved_csvs(file_dir=df_outage_dir, glob_pattern=r'outg_rec_nb_*.csv', 
                                                        cols_of_interest=cols_of_interest, 
                                                        verbose=True)
df_kwh_15T = kwh_vlt_dfs_dict['kwh']
df_vlt_15T = kwh_vlt_dfs_dict['vlt']

In [10]:
df_15T = df_kwh_15T.copy()

In [11]:
df_15T.head()

,serialnumber,prem_nb,srvc_pole_nb,trsf_pole_nb,annual_kwh,station_nb,xfmr_nb,starttimeperiod,endtimeperiod,aep_endtime_utc,timezoneoffset,aep_derived_uom,aep_srvc_qlty_idntfr,value,aep_usage_dt,aep_endtime_utc_from_timestamp,starttimeperiod_utc,endtimeperiod_utc
time_idx,,,,,,,,,,,,,,,,,,
2018-10-04 16:30:00,880693914,105148681,1832845704610,1832852704749,9043.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,TOTAL,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880693921,108248681,1832845704610,1832852704749,6530.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,TOTAL,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880687438,109826791,1832852704749,1832852704749,6078.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,TOTAL,0.810,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880687439,101436791,1832858704845,1832852704749,8022.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,TOTAL,0.584,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880693917,102048681,1832852704749,1832852704749,5851.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,TOTAL,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00


In [12]:
df_15T_OG = df_15T.copy()

In [13]:
df_15T = df_15T_OG.copy()

In [14]:
# df_15T_agg = df_15T.groupby(level=0).agg({'value':['mean', 'sum', 'count']})
# df_H_agg_1 = df_15T_agg.groupby(pd.Grouper(freq='H')).agg(['mean', 'sum', 'count'])
# #-----------------------------------
# df_H = df_15T.groupby([pd.Grouper(freq='H')] + ['serialnumber']).agg({'value':['mean', 'sum', 'count']})
# df_H_agg_2 = df_H.groupby(level=0).agg(['mean', 'sum', 'count'])

### Below 
- df_H_agg_1 is constructed by:
    - 1. Group collection of all meters, df_15T, by index = time (15T) to create df_15T_agg
        - which is the aggregate of all meters for each time period
    - 2. Resample df_15T_agg to hourly frequency to get df_H_agg_1
    
- df_H_agg_2 is constructed by:
    - 1. Resample collection of all meters, df_15T, by hour to create df_H
        - which contains hourly resampled data for each serial number individually
    - 2. Group df_H by index = time (hour) to get df_H_agg_2
    
    
FOR SOME REASON, df_H_agg_1 has a few more entries than df_H_agg_2 (182 more than 26039, or 0.69895% more)

IT APPEARS: When time resampling is done (whether via .resample(freq) or via .groupby(pd.Grouper(freq))), if there are 
<br>no entries within a given period, the period IS NOT OMITTED, but IS KEPT with all NaN values!
<br>BUT this is only true for PURE TIME RESAMPLING.  If, e.g. .groupby([pd.Grouper(freq='H')] + ['serialnumber']), then empty periods WILL BE OMITTED

In [15]:
df_15T = df_15T_OG.copy()

df_15T_agg = df_15T.groupby(level=0).agg({'value':['mean', 'sum', 'count']})
df_15T_agg.columns = df_15T_agg.columns.set_levels([f'{x}_mtrs' for x in df_15T_agg.columns.levels[-1]], level=-1)

df_H_agg_1 = df_15T_agg.groupby(pd.Grouper(freq='H')).agg(['mean', 'sum', 'count'])
df_H_agg_1.columns = df_H_agg_1.columns.set_levels([f'{x}_TRS' for x in df_H_agg_1.columns.levels[-1]], level=-1)
# #-----------------------------------
df_H = df_15T.groupby([pd.Grouper(freq='H')] + ['serialnumber']).agg({'value':['mean', 'sum', 'count']})
df_H.columns = df_H.columns.set_levels([f'{x}_TRS' for x in df_H.columns.levels[-1]], level=-1)

df_H_agg_2 = df_H.groupby(level=0).agg(['mean', 'sum', 'count'])
df_H_agg_2.columns = df_H_agg_2.columns.set_levels([f'{x}_mtrs' for x in df_H_agg_2.columns.levels[-1]], level=-1)

In [16]:
print(f'df_H_agg_1.shape = {df_H_agg_1.shape}')
print(f'df_H_agg_2.shape = {df_H_agg_2.shape}')

df_H_agg_1.shape = (26221, 9)
df_H_agg_2.shape = (26039, 9)


In [17]:
missing_idx = [x for x in df_H_agg_1.index if x not in df_H_agg_2.index]
print(f'len(missing_idx) = {len(missing_idx)}')

len(missing_idx) = 182


In [18]:
df_H_agg_1.head()

value                                               \
                    mean_mtrs                   sum_mtrs                     
                     mean_TRS sum_TRS count_TRS mean_TRS sum_TRS count_TRS   
time_idx                                                                     
2018-10-04 16:00:00    0.2764  0.5528         2   1.3820   2.764         2   
2018-10-04 17:00:00    0.2366  0.9464         4   1.1830   4.732         4   
2018-10-04 18:00:00    0.2933  1.1732         4   1.4665   5.866         4   
2018-10-04 19:00:00    0.4234  1.6936         4   2.1170   8.468         4   
2018-10-04 20:00:00    0.3280  1.3120         4   1.6400   6.560         4   

                                                  
                    count_mtrs                    
                      mean_TRS sum_TRS count_TRS  
time_idx                                          
2018-10-04 16:00:00        5.0      10         2  
2018-10-04 17:00:00        5.0      20         4  
2018-10-04 18:00:00        5.0      20         4  
2018-10-04 19:00:00        5.0      20         4  
2018-10-04 20:00:00        5.0      20         4

In [19]:
df_H_agg_2.head()

value                                         \
                     mean_TRS                       sum_TRS            
                    mean_mtrs sum_mtrs count_mtrs mean_mtrs sum_mtrs   
time_idx                                                               
2018-10-04 16:00:00    0.2764   1.3820          5    0.5528    2.764   
2018-10-04 17:00:00    0.2366   1.1830          5    0.9464    4.732   
2018-10-04 18:00:00    0.2933   1.4665          5    1.1732    5.866   
2018-10-04 19:00:00    0.4234   2.1170          5    1.6936    8.468   
2018-10-04 20:00:00    0.3280   1.6400          5    1.3120    6.560   

                                                              
                               count_TRS                      
                    count_mtrs mean_mtrs sum_mtrs count_mtrs  
time_idx                                                      
2018-10-04 16:00:00          5       2.0       10          5  
2018-10-04 17:00:00          5       4.0       20          5  
2018-10-04 18:00:00          5       4.0       20          5  
2018-10-04 19:00:00          5       4.0       20          5  
2018-10-04 20:00:00          5       4.0       20          5

In [20]:
# - df_H_agg_1 is constructed by:
#     - 1. Group collection of all meters, df_15T, by index = time (15T) to create df_15T_agg
#         - which is the aggregate of all meters for each time period
#     - 2. Resample df_15T_agg to hourly frequency to get df_H_agg_1

# 1_1: ('value',  'mean_mtrs',  'mean_TRS') - hourly mean of meters averaged over 15T
# 2_1: ('value',  'mean_mtrs',   'sum_TRS') - hourly sum of meters averaged over 15T
# 3_1: ('value',  'mean_mtrs', 'count_TRS') - number of 15T groupings with at least one meter in hour 
#                                               (should be 4, certainly no more than 4)

# 4_1: ('value',   'sum_mtrs',  'mean_TRS') - hourly mean of meters summed over 15T
# 5_1: ('value',   'sum_mtrs',   'sum_TRS') - hourly sum of meters summed over 15T
# 6_1: ('value',   'sum_mtrs', 'count_TRS') - number of 15T groupings with at least one meter in hour 
#                                              (should be 4, certainly no more than 4)

# 7_1: ('value', 'count_mtrs',  'mean_TRS') - hourly mean of number of 15T groupings with at least one meter in hour 
#                                              (most should be equal to max value, which should equal number of SNs)
# 8_1: ('value', 'count_mtrs',   'sum_TRS') - hourly sum of number of 15T groupings with at least one meter in hour 
#                                              (most should be equal to max value*4 (groupings per hour), which should equal number of SNs*4)
# 9_1: ('value', 'count_mtrs', 'count_TRS') - number of 15T groupings with at least one meter in hour 
#                                              (should be 4, certainly no more than 4)
    
# The following should be equal within df_H_agg_1:
#     3_1: ('value',  'mean_mtrs', 'count_TRS')
#     6_1: ('value',   'sum_mtrs', 'count_TRS')
#     9_1: ('value', 'count_mtrs', 'count_TRS')
#
# The following should be at least approximately equal...
#     8_1 ~ 7_1*9_1

#*************************************************************************************************************
# - df_H_agg_2 is constructed by:
#     - 1. Resample collection of all meters, df_15T, by hour to create df_H
#         - which contains hourly resampled data for each serial number individually
#     - 2. Group df_H by index = time (hour) to get df_H_agg_2

# 1_2: ('value',  'mean_TRS',  'mean_mtrs') - average over meters of meter hourly averages
# 2_2: ('value',  'mean_TRS',   'sum_mtrs') - sum over meters of meter hourly averages
# 3_2: ('value',  'mean_TRS', 'count_mtrs') - number of meters with at least one 15T value within hour
#                                               (most should be equal to max value, which should equal number of SNs)
    
# 4_2: ('value',   'sum_TRS',  'mean_mtrs') - average over meters of meter hourly sums
# 5_2: ('value',   'sum_TRS',   'sum_mtrs') - sum over meters of meter hourly sums
# 6_2: ('value',   'sum_TRS', 'count_mtrs') - number of meters with at least one 15T value within hour
#                                               (most should be equal to max value, which should equal number of SNs)
    
# 7_2: ('value', 'count_TRS',  'mean_mtrs') - mean number of meters with at least one 15T value in each hour
#                                               (should be approximately 4, certainly no more than 4)
# 8_2: ('value', 'count_TRS',   'sum_mtrs') - sum of number of meters with at least one 15T value in each hour
#                                               (most should be equal to 4*max value, which should equal 4*number of SNs)
# 9_2: ('value', 'count_TRS', 'count_mtrs') - number of meters with at least one 15T value within hour
#                                               (most should be equal to max value, which should equal number of SNs)
    
# The following should be equal within df_H_agg_2:
#     3_2: ('value',  'mean_TRS', 'count_mtrs')
#     6_2: ('value',   'sum_TRS', 'count_mtrs')
#     9_2: ('value', 'count_TRS', 'count_mtrs').
#
# The following should be at least approximately equal...
#     8_2 ~ 7_2*9_2

#*************************************************************************************************************
# The following should be equal between df_H_agg_1 and df_H_agg_2
# matching_cols_agg12 = {
#     ('value',  'mean_mtrs',  'mean_TRS') : ('value',  'mean_TRS',  'mean_mtrs'), 
#     ('value',  'mean_mtrs',   'sum_TRS') : ('value',   'sum_TRS',  'mean_mtrs'), 
#     ('value',   'sum_mtrs',  'mean_TRS') : ('value',  'mean_TRS',   'sum_mtrs'),
#     ('value',   'sum_mtrs',   'sum_TRS') : ('value',   'sum_TRS',   'sum_mtrs'),
# }

In [21]:
n_unique_sns = df_15T['serialnumber'].nunique()
print(f'n_unique_sns = {n_unique_sns}')

n_unique_sns = 6


In [22]:
print(f"3_1==6_1?: {df_H_agg_1[('value',  'mean_mtrs', 'count_TRS')].equals(df_H_agg_1[('value',   'sum_mtrs', 'count_TRS')])}")
print(f"3_1==9_1?: {df_H_agg_1[('value',  'mean_mtrs', 'count_TRS')].equals(df_H_agg_1[('value', 'count_mtrs', 'count_TRS')])}")

3_1==6_1?: True
3_1==9_1?: True


In [23]:
df_H_agg_1['7_1*9_1'] = df_H_agg_1[('value', 'count_mtrs',  'mean_TRS')]*df_H_agg_1[('value', 'count_mtrs', 'count_TRS')]
df_H_agg_1[('value', 'count_mtrs',   'sum_TRS')] = df_H_agg_1[('value', 'count_mtrs',   'sum_TRS')].astype(np.float64)
print(f"8_1=7_1*9_1?: {df_H_agg_1[('value', 'count_mtrs',   'sum_TRS')].equals(df_H_agg_1['7_1*9_1'])}")

8_1=7_1*9_1?: False


In [24]:
df_71_91_ne_81 = df_H_agg_1[df_H_agg_1[('value', 'count_mtrs',   'sum_TRS')] != df_H_agg_1['7_1*9_1']]
print(f"df_71_91_ne_81.shape[0]        = {df_71_91_ne_81.shape[0]}")
print(f"df_71_91_ne_81.dropna.shape[0] = {df_71_91_ne_81.dropna().shape[0]}")

df_71_91_ne_81.shape[0]        = 182
df_71_91_ne_81.dropna.shape[0] = 0


In [25]:
# In df_71_91_ne_81, all indices from 2019-01-02 06:00:00 through 2019-01-03 04:00:00 have all NaNs
#    NOTE: Others exist in df_71_91_ne_81, this is just an example
# df_15T.loc['2019-01-02 05:00:00':'2019-01-03 05:15:00']
df_15T_agg.loc['2019-01-02 04:00:00':'2019-01-03 05:15:00']

value                    
                    mean_mtrs sum_mtrs count_mtrs
time_idx                                         
2019-01-02 04:00:00  0.188667    1.132          6
2019-01-02 04:15:00  0.239000    1.434          6
2019-01-02 04:30:00  0.264333    1.586          6
2019-01-02 04:45:00  0.145000    0.870          6
2019-01-02 05:00:00  0.279667    1.678          6
2019-01-03 05:15:00  0.193000    1.158          6

In [26]:
# So, in the 15T data, there simply are no entries between '2019-01-02 05:00:00':'2019-01-03 05:15:00'
# When df_15T_agg is resampled
df_H_agg_1.loc['2019-01-02 04:00:00':'2019-01-03 05:15:00']

value                                                  \
                    mean_mtrs                      sum_mtrs                     
                     mean_TRS   sum_TRS count_TRS  mean_TRS sum_TRS count_TRS   
time_idx                                                                        
2019-01-02 04:00:00  0.209250  0.837000         4  1.255500   5.022         4   
2019-01-02 05:00:00  0.279667  0.279667         1  1.678000   1.678         1   
2019-01-02 06:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 07:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 08:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 09:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 10:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 11:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 12:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 13:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 14:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 15:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 16:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 17:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 18:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 19:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 20:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 21:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 22:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-02 23:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-03 00:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-03 01:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-03 02:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-03 03:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-03 04:00:00       NaN  0.000000         0       NaN   0.000         0   
2019-01-03 05:00:00  0.169111  0.507333         3  1.014667   3.044         3   

                                                 7_1*9_1  
                    count_mtrs                            
                      mean_TRS sum_TRS count_TRS          
time_idx                                                  
2019-01-02 04:00:00        6.0    24.0         4    24.0  
2019-01-02 05:00:00        6.0     6.0         1     6.0  
2019-01-02 06:00:00        NaN     0.0         0     NaN  
2019-01-02 07:00:00        NaN     0.0         0     NaN  
2019-01-02 08:00:00        NaN     0.0         0     NaN  
2019-01-02 09:00:00        NaN     0.0         0     NaN  
2019-01-02 10:00:00        NaN     0.0         0     NaN  
2019-01-02 11:00:00        NaN     0.0         0     NaN  
2019-01-02 12:00:00        NaN     0.0         0     NaN  
2019-01-02 13:00:00        NaN     0.0         0     NaN  
2019-01-02 14:00:00        NaN     0.0         0     NaN  
2019-01-02 15:00:00        NaN     0.0         0     NaN  
2019-01-02 16:00:00        NaN     0.0         0     NaN  
2019-01-02 17:00:00        NaN     0.0         0     NaN  
2019-01-02 18:00:00        NaN     0.0         0     NaN  
2019-01-02 19:00:00        NaN     0.0         0     NaN  
2019-01-02 20:00:00        NaN     0.0         0     NaN  
2019-01-02 21:00:00        NaN     0.0         0     NaN  
2019-01-02 22:00:00        NaN     0.0         0     NaN  
2019-01-02 23:00:00        NaN     0.0         0     NaN  
2019-01-03 00:00:00        NaN     0.0         0     NaN  
2019-01-03 01:00:00        NaN     0.0         0     NaN  
2019-01-03 02:00:00        NaN     0.0         0     NaN  


In [27]:
print("df_H_agg_1[('value', 'count_mtrs',  'mean_TRS')]")
maxv = df_H_agg_1[('value', 'count_mtrs',  'mean_TRS')].max()
pct_at_max = df_H_agg_1[df_H_agg_1[('value', 'count_mtrs',  'mean_TRS')]==maxv].shape[0]/df_H_agg_1.shape[0]
print(f'maxv              = {maxv}')
print(f'n_unique_sns      = {n_unique_sns}')
print(f'maxv==n_unique_sns: {maxv==n_unique_sns}')
print(f'pct_at_max        = {pct_at_max} (should be close to 1)')
#---------------------------------------------------------------------------------
print('\n\n')
print("df_H_agg_1[('value', 'count_mtrs',   'sum_TRS')]")
maxv = df_H_agg_1[('value', 'count_mtrs',  'sum_TRS')].max()
pct_at_max = df_H_agg_1[df_H_agg_1[('value', 'count_mtrs',  'sum_TRS')]==maxv].shape[0]/df_H_agg_1.shape[0]
print(f'maxv              = {maxv}')
print(f'n_unique_sns      = {n_unique_sns}')
print(f'maxv==4.0*n_unique_sns: {maxv==4.0*n_unique_sns}')
print(f'pct_at_max        = {pct_at_max} (should be close to 1)')
#---------------------------------------------------------------------------------
print('\n\n')
print("df_H_agg_1[('value',  'mean_mtrs', 'count_TRS')]")
maxv = df_H_agg_1[('value', 'mean_mtrs',  'count_TRS')].max()
pct_at_max = df_H_agg_1[df_H_agg_1[('value', 'mean_mtrs',  'count_TRS')]==maxv].shape[0]/df_H_agg_1.shape[0]
print(f'maxv              = {maxv}')
print(f'maxv==4:            {maxv==4}')
print(f'pct_at_max        = {pct_at_max} (should be close to 1)')

df_H_agg_1[('value', 'count_mtrs',  'mean_TRS')]
maxv              = 6.0
n_unique_sns      = 6
maxv==n_unique_sns: True
pct_at_max        = 0.9433278669768507 (should be close to 1)



df_H_agg_1[('value', 'count_mtrs',   'sum_TRS')]
maxv              = 24.0
n_unique_sns      = 6
maxv==4.0*n_unique_sns: True
pct_at_max        = 0.9422600205941802 (should be close to 1)



df_H_agg_1[('value',  'mean_mtrs', 'count_TRS')]
maxv              = 4
maxv==4:            True
pct_at_max        = 0.9907326188932535 (should be close to 1)


In [28]:
print(f"3_2==6_2?: {df_H_agg_2[('value',  'mean_TRS', 'count_mtrs')].equals(df_H_agg_2[('value',   'sum_TRS', 'count_mtrs')])}")
print(f"3_2==9_2?: {df_H_agg_2[('value',  'mean_TRS', 'count_mtrs')].equals(df_H_agg_2[('value', 'count_TRS', 'count_mtrs')])}")

3_2==6_2?: True
3_2==9_2?: True


In [29]:
df_H_agg_2['7_2*9_2'] = df_H_agg_2[('value', 'count_TRS',  'mean_mtrs')]*df_H_agg_2[('value', 'count_TRS', 'count_mtrs')]
df_H_agg_2[('value', 'count_TRS',   'sum_mtrs')] = df_H_agg_2[('value', 'count_TRS',   'sum_mtrs')].astype(np.float64)
print(f"8_2=7_2*9_2?: {df_H_agg_2[('value', 'count_TRS',   'sum_mtrs')].equals(df_H_agg_2['7_2*9_2'])}")

8_2=7_2*9_2?: True


In [30]:
df_72_92_ne_82 = df_H_agg_2[df_H_agg_2[('value', 'count_TRS',   'sum_mtrs')] != df_H_agg_2['7_2*9_2']]
print(f"df_72_92_ne_82.shape[0]        = {df_72_92_ne_82.shape[0]}")
print(f"df_72_92_ne_82.dropna.shape[0] = {df_72_92_ne_82.dropna().shape[0]}")

df_72_92_ne_82.shape[0]        = 0
df_72_92_ne_82.dropna.shape[0] = 0


In [31]:
print("df_H_agg_2[('value',  'mean_TRS', 'count_mtrs')]")
maxv = df_H_agg_2[('value',  'mean_TRS', 'count_mtrs')].max()
pct_at_max = df_H_agg_2[df_H_agg_2[('value',  'mean_TRS', 'count_mtrs')]==maxv].shape[0]/df_H_agg_2.shape[0]
print(f'maxv              = {maxv}')
print(f'n_unique_sns      = {n_unique_sns}')
print(f'maxv==n_unique_sns: {maxv==n_unique_sns}')
print(f'pct_at_max        = {pct_at_max} (should be close to 1)')
#---------------------------------------------------------------------------------

df_H_agg_2[('value',  'mean_TRS', 'count_mtrs')]
maxv              = 6
n_unique_sns      = 6
maxv==n_unique_sns: True
pct_at_max        = 0.9605975651906755 (should be close to 1)


In [32]:
#     ('value',  'mean_mtrs',  'mean_TRS') : ('value',  'mean_TRS',  'mean_mtrs'), 
#     ('value',  'mean_mtrs',   'sum_TRS') : ('value',   'sum_TRS',  'mean_mtrs'), 
#     ('value',   'sum_mtrs',  'mean_TRS') : ('value',  'mean_TRS',   'sum_mtrs'),
#     ('value',   'sum_mtrs',   'sum_TRS') : ('value',   'sum_TRS',   'sum_mtrs'),

In [33]:
df_H_agg_1['comp_col_1'] = df_H_agg_1[('value',  'mean_mtrs',  'mean_TRS')]
df_H_agg_1['comp_col_2'] = df_H_agg_1[('value',  'mean_mtrs',   'sum_TRS')]
df_H_agg_1['comp_col_3'] = df_H_agg_1[('value',   'sum_mtrs',  'mean_TRS')]
df_H_agg_1['comp_col_4'] = df_H_agg_1[('value',   'sum_mtrs',   'sum_TRS')]

In [34]:
df_H_agg_2['comp_col_1'] = df_H_agg_2[('value',  'mean_TRS',  'mean_mtrs')]
df_H_agg_2['comp_col_2'] = df_H_agg_2[('value',   'sum_TRS',  'mean_mtrs')]
df_H_agg_2['comp_col_3'] = df_H_agg_2[('value',  'mean_TRS',   'sum_mtrs')]
df_H_agg_2['comp_col_4'] = df_H_agg_2[('value',   'sum_TRS',   'sum_mtrs')]

In [35]:
comp_cols = ['comp_col_1', 'comp_col_2', 'comp_col_3', 'comp_col_4']
df_H_agg_1_subset = df_H_agg_1[comp_cols].dropna()
df_H_agg_2_subset = df_H_agg_2[comp_cols]

In [36]:
diffs = Utilities_df.get_dfs_diff_WEIRD(df_H_agg_1_subset, df_H_agg_2_subset, stack_level=0)
diffs

df1_values  df2_values
row                 col                               
2018-10-04 22:00:00 comp_col_1    0.242500    0.242500
                    comp_col_2    0.970000    0.970000
2018-10-05 02:00:00 comp_col_1    0.332600    0.332600
                    comp_col_2    1.330400    1.330400
2018-10-05 03:00:00 comp_col_1    0.185300    0.185300
...                                    ...         ...
2021-10-01 01:00:00 comp_col_2    0.643333    0.643333
2021-10-01 02:00:00 comp_col_1    0.114333    0.114333
                    comp_col_2    0.457333    0.457333
                    comp_col_3    0.686000    0.686000
                    comp_col_4    2.744000    2.744000

[28135 rows x 2 columns]

In [37]:
approx_diffs = Utilities_df.get_dfs_diff_approx_ok_WEIRD(df_H_agg_1_subset, df_H_agg_2_subset, stack_level=0)
approx_diffs

df1_values  df2_values  rel_delta
row                 col                                          
2018-10-10 16:00:00 comp_col_1    0.403450    0.366917   0.090552
                    comp_col_2    1.613800    1.467667   0.090552
2018-12-05 05:00:00 comp_col_1    0.156750    0.126917   0.190324
                    comp_col_2    0.627000    0.373667   0.404040
                    comp_col_3    0.560500    0.761500   0.358608
...                                    ...         ...        ...
2021-02-25 21:00:00 comp_col_2    0.285400    0.224333   0.213969
                    comp_col_3    0.448667    0.712000   0.586924
2021-02-25 23:00:00 comp_col_1    0.113000    0.121417   0.074484
                    comp_col_2    0.452000    0.429667   0.049410
                    comp_col_3    0.644500    0.728500   0.130334

[1300 rows x 3 columns]

# Look into differences using 2018-10-10 16:00:00

In [38]:
for sn in df_15T['serialnumber'].unique().tolist():
    print(df_15T[df_15T['serialnumber']==sn]['2018-10-10 16:00:00':'2018-10-10 16:59:00'][['serialnumber', 'value']])
    print()

                     serialnumber  value
time_idx                                
2018-10-10 16:00:00     880693914  0.686
2018-10-10 16:15:00     880693914  0.710
2018-10-10 16:30:00     880693914  0.698
2018-10-10 16:45:00     880693914  0.708

                     serialnumber  value
time_idx                                
2018-10-10 16:00:00     880693921  0.562
2018-10-10 16:15:00     880693921  0.560
2018-10-10 16:30:00     880693921  0.560
2018-10-10 16:45:00     880693921  0.540

                     serialnumber  value
time_idx                                
2018-10-10 16:00:00     880687438  0.110
2018-10-10 16:15:00     880687438  0.080
2018-10-10 16:30:00     880687438  0.104
2018-10-10 16:45:00     880687438  0.152

                     serialnumber  value
time_idx                                
2018-10-10 16:00:00     880687439  0.546
2018-10-10 16:15:00     880687439  0.534
2018-10-10 16:30:00     880687439  0.518
2018-10-10 16:45:00     880687439  0.524

            

In [39]:
df_15T_agg.loc['2018-10-10 16:00:00':'2018-10-10 16:59:00']

value                    
                    mean_mtrs sum_mtrs count_mtrs
time_idx                                         
2018-10-10 16:00:00    0.4672    2.336          5
2018-10-10 16:15:00    0.4096    2.048          5
2018-10-10 16:30:00    0.3570    2.142          6
2018-10-10 16:45:00    0.3800    2.280          6

In [40]:
df_H_agg_1.loc['2018-10-10 16:00:00']

value       mean_mtrs   mean_TRS      0.40345
                        sum_TRS       1.61380
                        count_TRS     4.00000
            sum_mtrs    mean_TRS      2.20150
                        sum_TRS       8.80600
                        count_TRS     4.00000
            count_mtrs  mean_TRS      5.50000
                        sum_TRS      22.00000
                        count_TRS     4.00000
7_1*9_1                              22.00000
comp_col_1                            0.40345
comp_col_2                            1.61380
comp_col_3                            2.20150
comp_col_4                            8.80600
Name: 2018-10-10 16:00:00, dtype: float64

In [41]:
df_H.loc['2018-10-10 16:00:00']

value                  
                                 mean_TRS sum_TRS count_TRS
time_idx            serialnumber                           
2018-10-10 16:00:00 880687438      0.1115   0.446         4
                    880687439      0.5305   2.122         4
                    880693914      0.7005   2.802         4
                    880693917      0.3035   1.214         4
                    880693921      0.5555   2.222         4
                    880713197      0.0000   0.000         2

In [42]:
df_H_agg_2.loc['2018-10-10 16:00:00']

value       mean_TRS   mean_mtrs      0.366917
                       sum_mtrs       2.201500
                       count_mtrs     6.000000
            sum_TRS    mean_mtrs      1.467667
                       sum_mtrs       8.806000
                       count_mtrs     6.000000
            count_TRS  mean_mtrs      3.666667
                       sum_mtrs      22.000000
                       count_mtrs     6.000000
7_2*9_2                              22.000000
comp_col_1                            0.366917
comp_col_2                            1.467667
comp_col_3                            2.201500
comp_col_4                            8.806000
Name: 2018-10-10 16:00:00, dtype: float64

# NEW FUNCS!

In [43]:
# def build_aggregate_of_meters_df():
#     # Previously was build_df_aggregated_for_each_time_index


# def build_time_resampled_df():
#     # Previously was build_resampled_df


# def build_time_resampled_aggregate_of_meters_df():
#     # Previously was build_resampled_df_aggregated_for_each_time_index
#     # First, perform aggregation of meters
#     # Then, time resample

In [44]:
#TODO THIS IS AN INTERESTING IDEAD
# Downside to _1 is that argument names always need ot be included
#   i.e. could call    build_TRS_df_1(df_15T=df_15T, freq=freq)
#        cold not call build_TRS_df_1(df_15T, freq)
# Uside: no need to update if build_time_resampled_df is updated
def build_TRS_df_1(**kwargs):
    return AMINonVee.build_time_resampled_df(**kwargs)

# May want to require any arguments without default values
# Downside to _2 : If params with default values get changed etc. in build_time_resampled_df
#                  This would need to be done accordingly.  This leaves room for human error as one can
#                  easily forget to do this
# Upside: can call e.g. build_TRS_df_1(df_15T, freq)
def build_TRS_df_2(df_15T, freq, **kwargs):
    return AMINonVee.build_time_resampled_df(df_15T, freq, **kwargs)

In [45]:
# df1 = AMINonVee.build_time_resampled_df(df_15T_aggNEW, 'H', other_grouper_cols=[], agg_cols=[('value',  'mean_mtrs')], agg_types=agg_types)
# df2 = build_TRS_df_1(df_15T=df_15T_aggNEW, freq='H', other_grouper_cols=[], agg_cols=[('value',  'mean_mtrs')], agg_types=agg_types)
# df3 = build_TRS_df_2(df_15T_aggNEW, 'H', other_grouper_cols=[], agg_cols=[('value',  'mean_mtrs')], agg_types=agg_types)

In [2]:
kwh_vlt_dfs_dict = assemble_kwh_vlt_dfs_from_saved_csvs(file_dir=df_outage_dir, glob_pattern=r'outg_rec_nb_*.csv', 
                                                        cols_of_interest=cols_of_interest, 
                                                        verbose=True, combine_kwh_delivered_and_received=False)
df_kwh_15T = kwh_vlt_dfs_dict['kwh']
df_vlt_15T = kwh_vlt_dfs_dict['vlt']

df_15T = df_kwh_15T.copy()
df_15T.head()

In [47]:
df_15T = df_kwh_15T.copy()

In [48]:
df_15T.head()

,serialnumber,prem_nb,srvc_pole_nb,trsf_pole_nb,annual_kwh,station_nb,xfmr_nb,starttimeperiod,endtimeperiod,aep_endtime_utc,timezoneoffset,aep_derived_uom,aep_srvc_qlty_idntfr,value,aep_usage_dt,aep_endtime_utc_from_timestamp,starttimeperiod_utc,endtimeperiod_utc
time_idx,,,,,,,,,,,,,,,,,,
2018-10-04 16:30:00,880693921,108248681,1832845704610,1832852704749,6530.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,RECEIVED,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880693917,102048681,1832852704749,1832852704749,5851.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,RECEIVED,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880687438,109826791,1832852704749,1832852704749,6078.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,RECEIVED,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880693921,108248681,1832845704610,1832852704749,6530.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,TOTAL,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880687439,101436791,1832858704845,1832852704749,8022.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,TOTAL,0.584,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00


In [49]:
df_15T_agg = df_15T.groupby(level=0).agg({'value':['mean', 'sum', 'count'], 'aep_srvc_qlty_idntfr':['first']})
df_15T_agg.columns = df_15T_agg.columns.set_levels([f'{x}_mtrs' for x in df_15T_agg.columns.levels[-1]], level=-1)

In [50]:
df_15T_agg

value                     aep_srvc_qlty_idntfr
                    mean_mtrs sum_mtrs count_mtrs           first_mtrs
time_idx                                                              
2018-10-04 16:30:00  0.139400    1.394         10             RECEIVED
2018-10-04 16:45:00  0.137000    1.370         10                TOTAL
2018-10-04 17:00:00  0.134600    1.346         10                TOTAL
2018-10-04 17:15:00  0.136200    1.362         10             RECEIVED
2018-10-04 17:30:00  0.110600    1.106         10             RECEIVED
...                       ...      ...        ...                  ...
2021-10-01 03:00:00  0.053167    0.638         12                TOTAL
2021-10-01 03:15:00  0.061500    0.738         12             RECEIVED
2021-10-01 03:30:00  0.067667    0.812         12                TOTAL
2021-10-01 03:45:00  0.089167    1.070         12                TOTAL
2021-10-01 04:00:00  0.051833    0.622         12                TOTAL

[104059 rows x 4 columns]

In [51]:
df_H_agg_1 = df_15T_agg.groupby([pd.Grouper(freq='H')] + [('aep_srvc_qlty_idntfr', 'first_mtrs')]).agg({
    ('value',  'mean_mtrs'):['mean', 'sum', 'count'], 
    ('value',  'sum_mtrs'):['mean', 'sum', 'count'], 
    ('value',  'count_mtrs'):['mean', 'sum', 'count']
})
df_H_agg_1.columns = df_H_agg_1.columns.set_levels([f'{x}_TRS' for x in df_H_agg_1.columns.levels[-1]], level=-1)

In [52]:
df_H_agg_1

value            \
                                                       mean_mtrs             
                                                        mean_TRS   sum_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                       
2018-10-04 16:00:00 RECEIVED                            0.139400  0.139400   
                    TOTAL                               0.137000  0.137000   
2018-10-04 17:00:00 RECEIVED                            0.112867  0.338600   
                    TOTAL                               0.134600  0.134600   
2018-10-04 18:00:00 RECEIVED                            0.168100  0.336200   
...                                                          ...       ...   
2021-10-01 02:00:00 RECEIVED                            0.061333  0.122667   
                    TOTAL                               0.053000  0.106000   
2021-10-01 03:00:00 RECEIVED                            0.061500  0.061500   
                    TOTAL                               0.070000  0.210000   
2021-10-01 04:00:00 TOTAL                               0.051833  0.051833   

                                                                            \
                                                                  sum_mtrs   
                                                       count_TRS  mean_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                       
2018-10-04 16:00:00 RECEIVED                                   1  1.394000   
                    TOTAL                                      1  1.370000   
2018-10-04 17:00:00 RECEIVED                                   3  1.128667   
                    TOTAL                                      1  1.346000   
2018-10-04 18:00:00 RECEIVED                                   2  1.681000   
...                                                          ...       ...   
2021-10-01 02:00:00 RECEIVED                                   2  0.736000   
                    TOTAL                                      2  0.636000   
2021-10-01 03:00:00 RECEIVED                                   1  0.738000   
                    TOTAL                                      3  0.840000   
2021-10-01 04:00:00 TOTAL                                      1  0.622000   

                                                                          \
                                                                           
                                                       sum_TRS count_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                     
2018-10-04 16:00:00 RECEIVED                             1.394         1   
                    TOTAL                                1.370         1   
2018-10-04 17:00:00 RECEIVED                             3.386         3   
                    TOTAL                                1.346         1   
2018-10-04 18:00:00 RECEIVED                             3.362         2   
...                                                        ...       ...   
2021-10-01 02:00:00 RECEIVED                             1.472         2   
                    TOTAL                                1.272         2   
2021-10-01 03:00:00 RECEIVED                             0.738         1   
                    TOTAL                                2.520         3   
2021-10-01 04:00:00 TOTAL                                0.622         1   

                                                                           \
                                                       count_mtrs           
                                                         mean_TRS sum_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                      
2018-10-04 16:00:00 RECEIVED                                 10.0      10   
                    TOTAL                                    10.0      10   
2018-10-04 17:00:00 RECEIVED                                 10.0      30   
                    TOTAL            

In [53]:
df_H_agg_1[df_H_agg_1[('value', 'count_mtrs', 'mean_TRS')]%1!=0]

value            \
                                                       mean_mtrs             
                                                        mean_TRS   sum_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                       
2018-12-05 05:00:00 RECEIVED                            0.078375  0.313500   
2019-04-28 04:00:00 TOTAL                               0.066200  0.264800   
2019-07-14 00:00:00 RECEIVED                            0.240167  0.720500   
2019-08-03 04:00:00 TOTAL                               0.109606  0.219212   
2019-08-03 06:00:00 RECEIVED                            0.094162  0.282485   
...                                                          ...       ...   
2020-05-16 04:00:00 RECEIVED                            0.048944  0.146833   
2020-05-18 23:00:00 RECEIVED                            0.054333  0.163000   
2020-07-10 12:00:00 RECEIVED                            0.015889  0.047667   
2020-07-13 20:00:00 RECEIVED                            0.241233  0.723700   
2020-07-14 12:00:00 TOTAL                               0.027711  0.083133   

                                                                            \
                                                                  sum_mtrs   
                                                       count_TRS  mean_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                       
2018-12-05 05:00:00 RECEIVED                                   4  0.560500   
2019-04-28 04:00:00 TOTAL                                      4  0.699500   
2019-07-14 00:00:00 RECEIVED                                   3  0.585333   
2019-08-03 04:00:00 TOTAL                                      2  1.275000   
2019-08-03 06:00:00 RECEIVED                                   3  1.100000   
...                                                          ...       ...   
2020-05-16 04:00:00 RECEIVED                                   3  0.352000   
2020-05-18 23:00:00 RECEIVED                                   3  0.474667   
2020-07-10 12:00:00 RECEIVED                                   3  0.156000   
2020-07-13 20:00:00 RECEIVED                                   3  2.516000   
2020-07-14 12:00:00 TOTAL                                      3  0.312000   

                                                                          \
                                                                           
                                                       sum_TRS count_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                     
2018-12-05 05:00:00 RECEIVED                             2.242         4   
2019-04-28 04:00:00 TOTAL                                2.798         4   
2019-07-14 00:00:00 RECEIVED                             1.756         3   
2019-08-03 04:00:00 TOTAL                                2.550         2   
2019-08-03 06:00:00 RECEIVED                             3.300         3   
...                                                        ...       ...   
2020-05-16 04:00:00 RECEIVED                             1.056         3   
2020-05-18 23:00:00 RECEIVED                             1.424         3   
2020-07-10 12:00:00 RECEIVED                             0.468         3   
2020-07-13 20:00:00 RECEIVED                             7.548         3   
2020-07-14 12:00:00 TOTAL                                0.936         3   

                                                                           \
                                                       count_mtrs           
                                                         mean_TRS sum_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                      
2018-12-05 05:00:00 RECEIVED                             7.500000      30   
2019-04-28 04:00:00 TOTAL                               10.500000      42   
2019-07-14 00:00:00 RECEIVED                             2.666667       8   
2019-08-03 04:00:00 TOTAL            

In [54]:
df_15T_agg['2018-12-05 05:00:00':'2018-12-05 05:59:59']

value                     aep_srvc_qlty_idntfr
                    mean_mtrs sum_mtrs count_mtrs           first_mtrs
time_idx                                                              
2018-12-05 05:00:00  0.060167    0.722         12             RECEIVED
2018-12-05 05:15:00  0.091000    0.546          6             RECEIVED
2018-12-05 05:30:00  0.081667    0.490          6             RECEIVED
2018-12-05 05:45:00  0.080667    0.484          6             RECEIVED

In [55]:
# df_15T_agg = df_15T.groupby(level=0).agg({'value':['mean', 'sum', 'count'], 'aep_srvc_qlty_idntfr':['first']})
# df_15T_agg.columns = df_15T_agg.columns.set_levels([f'{x}_mtrs' for x in df_15T_agg.columns.levels[-1]], level=-1)

# df_H_agg_1 = df_15T_agg.groupby(pd.Grouper(freq='H')).agg(['mean', 'sum', 'count'])
# df_H_agg_1.columns = df_H_agg_1.columns.set_levels([f'{x}_TRS' for x in df_H_agg_1.columns.levels[-1]], level=-1)
# # #-----------------------------------
# df_H = df_15T.groupby([pd.Grouper(freq='H')] + ['serialnumber']).agg({'value':['mean', 'sum', 'count']})
# df_H.columns = df_H.columns.set_levels([f'{x}_TRS' for x in df_H.columns.levels[-1]], level=-1)

# df_H_agg_2 = df_H.groupby(level=0).agg(['mean', 'sum', 'count'])
# df_H_agg_2.columns = df_H_agg_2.columns.set_levels([f'{x}_mtrs' for x in df_H_agg_2.columns.levels[-1]], level=-1)

In [56]:
path_agg_cir = os.path.join(Utilities.get_local_data_dir(), r'sample_circuits\NewMethod\outg_rec_nb_11751094\outg_rec_nb_11751094_2020_q4.csv')
df_agg_cir = pd.read_csv(path_agg_cir)
df_agg_cir = Utilities_df.remove_prepend_from_columns_in_df(df_agg_cir)

In [57]:
path_agg_xfmr = os.path.join(Utilities.get_local_data_dir(), r'sample_circuits\NewMethod\GroupByXfmr\outg_rec_nb_11751094\outg_rec_nb_11751094_2020_q4.csv')
df_agg_xfmr = pd.read_csv(path_agg_xfmr)
df_agg_xfmr = df_agg_xfmr[df_agg_xfmr['aep_derived_uom']=='KWH']

In [58]:
df_agg_xfmr.head()

,trsf_pole_nb,starttimeperiod,endtimeperiod,aep_endtime_utc,timezoneoffset,aep_derived_uom,aep_srvc_qlty_idntfr,aep_usage_dt,value_sum,value_sq_sum,value_mean,value_std,counts,counts_including_null
2,1826288709850,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,4.110,6.328746,1.370,0.590782,3,3
6,1826288709850,2020-10-01T00:15:00-04:00,2020-10-01T00:30:00-04:00,1601526600,-04:00,KWH,TOTAL,2020-10-01,0.081,0.006561,0.081,NaN,1,1
10,1826288709850,2020-10-01T00:30:00-04:00,2020-10-01T00:45:00-04:00,1601527500,-04:00,KWH,RECEIVED,2020-10-01,0.000,0.000000,0.000,0.000000,4,4
16,1826288709850,2020-10-01T00:45:00-04:00,2020-10-01T01:00:00-04:00,1601528400,-04:00,KWH,RECEIVED,2020-10-01,0.000,0.000000,0.000,0.000000,4,4
19,1826288709850,2020-10-01T01:00:00-04:00,2020-10-01T01:15:00-04:00,1601529300,-04:00,KWH,DELIVERED,2020-10-01,3.798,5.354118,1.266,0.522422,3,3


In [59]:
df_agg_xfmr['trsf_pole_nb'].nunique()

223

In [60]:
df_agg_xfmr[df_agg_xfmr['starttimeperiod']=='2020-10-01T00:00:00-04:00']

,trsf_pole_nb,starttimeperiod,endtimeperiod,aep_endtime_utc,timezoneoffset,aep_derived_uom,aep_srvc_qlty_idntfr,aep_usage_dt,value_sum,value_sq_sum,value_mean,value_std,counts,counts_including_null
2,1826288709850,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,4.110,6.328746,1.370000,0.590782,3,3
97108,1826537709239,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,TOTAL,2020-10-01,3.904,1.636080,0.139429,0.201085,28,28
114868,1826540708984,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,TOTAL,2020-10-01,1.980,1.414784,0.247500,0.363462,8,8
132458,1826557709559,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,TOTAL,2020-10-01,5.666,3.538916,0.166647,0.280405,34,34
149942,1826630708864,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,RECEIVED,2020-10-01,0.000,0.000000,0.000000,0.000000,14,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8762076,1833829703474,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,TOTAL,2020-10-01,1.070,0.463756,0.178333,0.233640,6,6
8779968,1833920703468,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,TOTAL,2020-10-01,1.402,1.104908,0.233667,0.394286,6,6
8797554,1834012703260,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,TOTAL,2020-10-01,1.066,0.223372,0.133250,0.107788,8,9
8815164,1834037703464,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,RECEIVED,2020-10-01,0.000,0.000000,0.000000,0.000000,6,6


In [61]:
df_agg_xfmr[(df_agg_xfmr['starttimeperiod']=='2020-10-01T00:00:00-04:00') & (df_agg_xfmr['trsf_pole_nb']==1828083708502)]

,trsf_pole_nb,starttimeperiod,endtimeperiod,aep_endtime_utc,timezoneoffset,aep_derived_uom,aep_srvc_qlty_idntfr,aep_usage_dt,value_sum,value_sq_sum,value_mean,value_std,counts,counts_including_null
957453,1828083708502,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,2.365,3.083449,0.788333,0.780718,3,3
5376291,1828083708502,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,RECEIVED,2020-10-01,0.000,0.000000,0.000000,0.000000,5,5
5376292,1828083708502,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,TOTAL,2020-10-01,0.444,0.099720,0.222000,0.033941,2,2


In [62]:
gpby_cols = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
             'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'aep_usage_dt']
agg_dict = {'value_sum':['mean', 'sum', 'count'], 
            'value_sq_sum':['mean', 'sum', 'count'], 
            'value_mean':['mean', 'sum', 'count'], 
            'value_std':['mean', 'sum', 'count'], 
            'counts':['mean', 'sum', 'count']}
df_agg2_xfmr_cir_v2 = df_agg_xfmr.groupby(gpby_cols).agg(agg_dict)
df_agg2_xfmr_cir_v2 = df_agg2_xfmr_cir_v2.reset_index()

In [63]:
df_agg2_xfmr_cir_v2[[df_agg2_xfmr_cir_v2.columns[0]]+df_agg2_xfmr_cir_v2.columns[6:].tolist()]

starttimeperiod aep_usage_dt value_sum                    \
                                                   mean         sum count   
0      2020-10-01T00:00:00-04:00   2020-10-01  3.649960   91.248999    25   
1      2020-10-01T00:00:00-04:00   2020-10-01  0.000000    0.000000   223   
2      2020-10-01T00:00:00-04:00   2020-10-01  1.262145  261.264000   207   
3      2020-10-01T00:15:00-04:00   2020-10-01  3.799720   94.992999    25   
4      2020-10-01T00:15:00-04:00   2020-10-01  0.000000    0.000000   223   
...                          ...          ...       ...         ...   ...   
26503  2020-12-31T23:30:00-05:00   2020-12-31  0.000000    0.000000   223   
26504  2020-12-31T23:30:00-05:00   2020-12-31  2.180454  451.354000   207   
26505  2020-12-31T23:45:00-05:00   2020-12-31  5.052577  131.367001    26   
26506  2020-12-31T23:45:00-05:00   2020-12-31  0.000000    0.000000   223   
26507  2020-12-31T23:45:00-05:00   2020-12-31  2.115386  437.885000   207   

      value_sq_sum                    value_mean                   value_std  \
              mean          sum count       mean         sum count      mean   
0        19.650849   491.271228    25   2.591101   64.777533    25  1.115588   
1         0.000000     0.000000   223   0.000000    0.000000   223  0.000000   
2         0.527996   109.295267   207   0.193565   40.067967   207  0.150198   
3        22.695966   567.399159    25   2.755656   68.891400    25  1.149565   
4         0.000000     0.000000   223   0.000000    0.000000   223  0.000000   
...            ...          ...   ...        ...         ...   ...       ...   
26503     0.000000     0.000000   223   0.000000    0.000000   223  0.000000   
26504     1.379632   285.583820   207   0.327556   67.804003   207  0.229947   
26505    49.986603  1299.651672    26   4.082385  106.142001    26  1.244037   
26506     0.000000     0.000000   223   0.000000    0.000000   223  0.000000   
26507     1.278667   264.684166   207   0.323284   66.919738   207  0.219383   

                          counts              
             sum count      mean   sum count  
0       7.809117     7  1.720000    43    25  
1       0.000000   190  7.556054  1685   223  
2      27.185798   181  7.937198  1643   207  
3       8.046955     7  1.720000    43    25  
4       0.000000   190  7.556054  1685   223  
...          ...   ...       ...   ...   ...  
26503   0.000000   190  7.578475  1690   223  
26504  41.620479   181  7.956522  1647   207  
26505   8.708260     7  1.692308    44    26  
26506   0.000000   190  7.578475  1690   223  
26507  39.708245   181  7.956522  1647   207  

[26508 rows x 17 columns]

In [64]:
print(df_agg_xfmr[(df_agg_xfmr['starttimeperiod']=='2020-10-01T00:00:00-04:00') & 
            (df_agg_xfmr['aep_srvc_qlty_idntfr']=='DELIVERED')].shape)

(25, 14)


In [65]:
df_agg_xfmr[(df_agg_xfmr['starttimeperiod']=='2020-10-01T00:00:00-04:00') & 
            (df_agg_xfmr['aep_srvc_qlty_idntfr']=='DELIVERED')]['counts'].sum()

43

In [66]:
df_agg_xfmr[(df_agg_xfmr['starttimeperiod']=='2020-10-01T00:00:00-04:00') & 
            (df_agg_xfmr['aep_srvc_qlty_idntfr']=='DELIVERED')]

,trsf_pole_nb,starttimeperiod,endtimeperiod,aep_endtime_utc,timezoneoffset,aep_derived_uom,aep_srvc_qlty_idntfr,aep_usage_dt,value_sum,value_sq_sum,value_mean,value_std,counts,counts_including_null
2,1826288709850,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,4.110,6.328746,1.370000,0.590782,3,3
379766,1827203709357,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,1.352,1.827904,1.352000,NaN,1,1
441228,1827231702675,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,4.692,22.014863,4.692000,NaN,1,1
520588,1827261707284,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,3.471,6.416360,0.694200,1.000849,5,5
886671,1827900707730,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,0.276,0.032976,0.092000,0.061579,3,3
957453,1828083708502,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,2.365,3.083449,0.788333,0.780718,3,3
1023798,1828104708349,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,0.888,0.788544,0.888000,NaN,1,1
1146450,1828154709883,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,8.160,66.585598,8.160000,NaN,1,1
1270452,1828326707030,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,9.612,46.217744,4.806000,0.149907,2,2
1394209,1828410704522,2020-10-01T00:00:00-04:00,2020-10-01T00:15:00-04:00,1601525700,-04:00,KWH,DELIVERED,2020-10-01,6.944,48.219133,6.944000,NaN,1,1


In [67]:
df_H_agg_1[df_H_agg_1[('value', 'count_mtrs', 'mean_TRS')]%1!=0]

value            \
                                                       mean_mtrs             
                                                        mean_TRS   sum_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                       
2018-12-05 05:00:00 RECEIVED                            0.078375  0.313500   
2019-04-28 04:00:00 TOTAL                               0.066200  0.264800   
2019-07-14 00:00:00 RECEIVED                            0.240167  0.720500   
2019-08-03 04:00:00 TOTAL                               0.109606  0.219212   
2019-08-03 06:00:00 RECEIVED                            0.094162  0.282485   
...                                                          ...       ...   
2020-05-16 04:00:00 RECEIVED                            0.048944  0.146833   
2020-05-18 23:00:00 RECEIVED                            0.054333  0.163000   
2020-07-10 12:00:00 RECEIVED                            0.015889  0.047667   
2020-07-13 20:00:00 RECEIVED                            0.241233  0.723700   
2020-07-14 12:00:00 TOTAL                               0.027711  0.083133   

                                                                            \
                                                                  sum_mtrs   
                                                       count_TRS  mean_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                       
2018-12-05 05:00:00 RECEIVED                                   4  0.560500   
2019-04-28 04:00:00 TOTAL                                      4  0.699500   
2019-07-14 00:00:00 RECEIVED                                   3  0.585333   
2019-08-03 04:00:00 TOTAL                                      2  1.275000   
2019-08-03 06:00:00 RECEIVED                                   3  1.100000   
...                                                          ...       ...   
2020-05-16 04:00:00 RECEIVED                                   3  0.352000   
2020-05-18 23:00:00 RECEIVED                                   3  0.474667   
2020-07-10 12:00:00 RECEIVED                                   3  0.156000   
2020-07-13 20:00:00 RECEIVED                                   3  2.516000   
2020-07-14 12:00:00 TOTAL                                      3  0.312000   

                                                                          \
                                                                           
                                                       sum_TRS count_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                     
2018-12-05 05:00:00 RECEIVED                             2.242         4   
2019-04-28 04:00:00 TOTAL                                2.798         4   
2019-07-14 00:00:00 RECEIVED                             1.756         3   
2019-08-03 04:00:00 TOTAL                                2.550         2   
2019-08-03 06:00:00 RECEIVED                             3.300         3   
...                                                        ...       ...   
2020-05-16 04:00:00 RECEIVED                             1.056         3   
2020-05-18 23:00:00 RECEIVED                             1.424         3   
2020-07-10 12:00:00 RECEIVED                             0.468         3   
2020-07-13 20:00:00 RECEIVED                             7.548         3   
2020-07-14 12:00:00 TOTAL                                0.936         3   

                                                                           \
                                                       count_mtrs           
                                                         mean_TRS sum_TRS   
time_idx            (aep_srvc_qlty_idntfr, first_mtrs)                      
2018-12-05 05:00:00 RECEIVED                             7.500000      30   
2019-04-28 04:00:00 TOTAL                               10.500000      42   
2019-07-14 00:00:00 RECEIVED                             2.666667       8   
2019-08-03 04:00:00 TOTAL            

# OLD VS NEW

In [68]:
def_15T = df_15T_OG.copy()

time_col_for_agg = 'time_idx'
agg_cols = ['value']
agg_types = ['mean', 'sum', 'count']

other_cols_to_keep = ['timezoneoffset', 'aep_derived_uom']

df_15T_agg_this_nb = df_15T.groupby(level=0).agg({'value':['mean', 'sum', 'count']})
df_15T_agg_this_nb.columns = df_15T_agg_this_nb.columns.set_levels([f'{x}_mtrs' for x in df_15T_agg_this_nb.columns.levels[-1]], level=-1)
#-----
df_15T_agg_old = build_df_aggregated_for_each_time_index(df_15T, time_col_for_agg, agg_cols, agg_types, 
                                                         other_cols_to_keep=other_cols_to_keep)
#-----
df_15T_agg_new = AMINonVee.build_aggregate_of_meters_df(df_15T, time_col_for_agg, agg_cols, agg_types, 
                                                            other_cols_to_keep=other_cols_to_keep)
df_15T_agg_new_unflat = AMINonVee.build_aggregate_of_meters_df(df_15T, time_col_for_agg, agg_cols, agg_types, 
                                                            other_cols_to_keep=other_cols_to_keep, flatten_columns=False)
#-----
# print(df_15T_agg_new.equals(Utilities_df.flatten_multiindex_columns(df_15T_agg_this_nb)))
# print(df_15T_agg_old.equals(Utilities_df.flatten_multiindex_columns(df_15T_agg_this_nb)))

In [69]:
print(df_15T_agg_old.equals(df_15T_agg_new))

True


In [70]:
df_H_agg_old = build_resampled_df(df_15T_agg_old, 'H', other_grouper_cols=[], agg_cols=['mean_mtrs value', 'sum_mtrs value', 'count_mtrs value'], agg_types=agg_types)
df_H_agg_new = AMINonVee.build_time_resampled_df(df_15T_agg_new, 'H', other_grouper_cols=[], agg_cols=['mean_mtrs value', 'sum_mtrs value', 'count_mtrs value'], agg_types=agg_types)
df_H_agg_new_unflat = AMINonVee.build_time_resampled_df(df_15T_agg_new, 'H', other_grouper_cols=[], agg_cols=['mean_mtrs value', 'sum_mtrs value', 'count_mtrs value'], agg_types=agg_types, flatten_columns=False)

In [71]:
df_H_agg_old.equals(df_H_agg_new)

True

In [72]:
df_H_agg_new

,mean_TRS mean_mtrs value,sum_TRS mean_mtrs value,count_TRS mean_mtrs value,mean_TRS sum_mtrs value,sum_TRS sum_mtrs value,count_TRS sum_mtrs value,mean_TRS count_mtrs value,sum_TRS count_mtrs value,count_TRS count_mtrs value
time_idx,,,,,,,,,
2018-10-04 16:00:00,0.138200,0.276400,2,1.3820,2.764,2,10.0,20,2
2018-10-04 17:00:00,0.118300,0.473200,4,1.1830,4.732,4,10.0,40,4
2018-10-04 18:00:00,0.146650,0.586600,4,1.4665,5.866,4,10.0,40,4
2018-10-04 19:00:00,0.211700,0.846800,4,2.1170,8.468,4,10.0,40,4
2018-10-04 20:00:00,0.164000,0.656000,4,1.6400,6.560,4,10.0,40,4
...,...,...,...,...,...,...,...,...,...
2021-10-01 00:00:00,0.098375,0.393500,4,1.1805,4.722,4,12.0,48,4
2021-10-01 01:00:00,0.080417,0.321667,4,0.9650,3.860,4,12.0,48,4
2021-10-01 02:00:00,0.057167,0.228667,4,0.6860,2.744,4,12.0,48,4


In [73]:
agg_cols = ['value']
agg_types = ['mean', 'sum', 'count']
mix_agg_functions=True

n_rounds=2
identifiers = ['_mtrs', '_TRS']

# n_rounds=3
# identifiers = ['_mtrs', '_TRS', 'ballz']

join_str = ' ' 
reverse_order=True 
to_ignore=['first']

all_rounds_agg_dicts, all_rounds_rename_dicts = AMINonVee.get_agg_and_rename_dicts_for_agg_rounds(agg_cols=agg_cols, agg_types=agg_types, 
                                                          mix_agg_functions=mix_agg_functions, n_rounds=n_rounds, identifiers=identifiers, 
                                                          join_str=join_str, reverse_order=reverse_order, to_ignore=to_ignore)
print(all_rounds_rename_dicts)
print()
print(all_rounds_agg_dicts)

{1: {('value', 'mean_mtrs'): 'mean_mtrs value', ('value', 'sum_mtrs'): 'sum_mtrs value', ('value', 'count_mtrs'): 'count_mtrs value'}, 2: {('mean_mtrs value', 'mean_TRS'): 'mean_TRS mean_mtrs value', ('mean_mtrs value', 'sum_TRS'): 'sum_TRS mean_mtrs value', ('mean_mtrs value', 'count_TRS'): 'count_TRS mean_mtrs value', ('sum_mtrs value', 'mean_TRS'): 'mean_TRS sum_mtrs value', ('sum_mtrs value', 'sum_TRS'): 'sum_TRS sum_mtrs value', ('sum_mtrs value', 'count_TRS'): 'count_TRS sum_mtrs value', ('count_mtrs value', 'mean_TRS'): 'mean_TRS count_mtrs value', ('count_mtrs value', 'sum_TRS'): 'sum_TRS count_mtrs value', ('count_mtrs value', 'count_TRS'): 'count_TRS count_mtrs value'}}

{1: {'value': ['mean', 'sum', 'count']}, 2: {'mean_mtrs value': ['mean', 'sum', 'count'], 'sum_mtrs value': ['mean', 'sum', 'count'], 'count_mtrs value': ['mean', 'sum', 'count']}}


In [74]:
df_15T_agg_test = df_15T_agg_new.copy()
time_col_for_meter_agg = 'time_idx'
agg_cols = ['value']
agg_types = ['mean', 'sum', 'count']

ugh_df_H_agg_old = build_resampled_df_aggregated_for_each_time_index(df_15T_agg_test, 
                                                                        agg_cols=agg_cols, agg_types=agg_types, 
                                                                        other_cols_to_keep=other_cols_to_keep, freq='H')

time_col_for_meter_agg = 'time_idx'
agg_cols = ['value']
agg_types = ['mean', 'sum', 'count']
agg_dict_meter_agg = None
agg_dict_TRS = None
other_cols_to_keep = ['timezoneoffset', 'aep_derived_uom']
mix_agg_functions=True
ugh_df_H_agg_newest = AMINonVee.build_time_resampled_aggregate_of_meters_df(df_15T, time_col_for_meter_agg=time_col_for_meter_agg, 
                                                                  agg_cols=agg_cols, agg_types=agg_types, 
                                                                  agg_dict_meter_agg=agg_dict_meter_agg, agg_dict_TRS=agg_dict_TRS, 
                                                                  other_cols_to_keep=other_cols_to_keep, freq='H', mix_agg_functions=mix_agg_functions)
print(ugh_df_H_agg_old.equals(ugh_df_H_agg_newest))

True


In [75]:
ugh_df_H_agg_old.head()

,mean_TRS mean_mtrs value,sum_TRS mean_mtrs value,count_TRS mean_mtrs value,mean_TRS sum_mtrs value,sum_TRS sum_mtrs value,count_TRS sum_mtrs value,mean_TRS count_mtrs value,sum_TRS count_mtrs value,count_TRS count_mtrs value,timezoneoffset,aep_derived_uom
time_idx,,,,,,,,,,,
2018-10-04 16:00:00,0.13820,0.2764,2,1.3820,2.764,2,10.0,20,2,-1 days +20:00:00,KWH
2018-10-04 17:00:00,0.11830,0.4732,4,1.1830,4.732,4,10.0,40,4,-1 days +20:00:00,KWH
2018-10-04 18:00:00,0.14665,0.5866,4,1.4665,5.866,4,10.0,40,4,-1 days +20:00:00,KWH
2018-10-04 19:00:00,0.21170,0.8468,4,2.1170,8.468,4,10.0,40,4,-1 days +20:00:00,KWH
2018-10-04 20:00:00,0.16400,0.6560,4,1.6400,6.560,4,10.0,40,4,-1 days +20:00:00,KWH


In [76]:
time_col_for_meter_agg = 'time_idx'
agg_cols = None
agg_types = None
agg_dict_meter_agg = {'value':['mean', 'sum', 'count']}
agg_dict_TRS={'mean_mtrs value': ['mean', 'sum', 'count'], 'sum_mtrs value': ['mean', 'sum', 'count'], 'count_mtrs value': ['mean', 'sum', 'count']}
other_cols_to_keep = ['timezoneoffset', 'aep_derived_uom']
mix_agg_functions=True
ugh_df_H_agg_newest2 = AMINonVee.build_time_resampled_aggregate_of_meters_df(df_15T, time_col_for_meter_agg=time_col_for_meter_agg, 
                                                                  agg_cols=agg_cols, agg_types=agg_types, 
                                                                   agg_dict_meter_agg=agg_dict_meter_agg, agg_dict_TRS=agg_dict_TRS, 
                                                                  other_cols_to_keep=other_cols_to_keep, freq='H', mix_agg_functions=mix_agg_functions)
print(ugh_df_H_agg_old.equals(ugh_df_H_agg_newest))

True


In [77]:
df_15T.head()

,serialnumber,prem_nb,srvc_pole_nb,trsf_pole_nb,annual_kwh,station_nb,xfmr_nb,starttimeperiod,endtimeperiod,aep_endtime_utc,timezoneoffset,aep_derived_uom,aep_srvc_qlty_idntfr,value,aep_usage_dt,aep_endtime_utc_from_timestamp,starttimeperiod_utc,endtimeperiod_utc
time_idx,,,,,,,,,,,,,,,,,,
2018-10-04 16:30:00,880693921,108248681,1832845704610,1832852704749,6530.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,RECEIVED,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880693917,102048681,1832852704749,1832852704749,5851.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,RECEIVED,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880687438,109826791,1832852704749,1832852704749,6078.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,RECEIVED,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880693921,108248681,1832845704610,1832852704749,6530.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,TOTAL,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880687439,101436791,1832858704845,1832852704749,8022.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,TOTAL,0.584,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00


In [78]:
ugh_df_H_agg_newest.head()

,mean_TRS mean_mtrs value,sum_TRS mean_mtrs value,count_TRS mean_mtrs value,mean_TRS sum_mtrs value,sum_TRS sum_mtrs value,count_TRS sum_mtrs value,mean_TRS count_mtrs value,sum_TRS count_mtrs value,count_TRS count_mtrs value,timezoneoffset,aep_derived_uom
time_idx,,,,,,,,,,,
2018-10-04 16:00:00,0.13820,0.2764,2,1.3820,2.764,2,10.0,20,2,-1 days +20:00:00,KWH
2018-10-04 17:00:00,0.11830,0.4732,4,1.1830,4.732,4,10.0,40,4,-1 days +20:00:00,KWH
2018-10-04 18:00:00,0.14665,0.5866,4,1.4665,5.866,4,10.0,40,4,-1 days +20:00:00,KWH
2018-10-04 19:00:00,0.21170,0.8468,4,2.1170,8.468,4,10.0,40,4,-1 days +20:00:00,KWH
2018-10-04 20:00:00,0.16400,0.6560,4,1.6400,6.560,4,10.0,40,4,-1 days +20:00:00,KWH


In [79]:
time_col_for_meter_agg = 'time_idx'
agg_cols = ['value']
agg_types = ['mean', 'sum', 'count']
agg_dict_meter_agg = None
agg_dict_TRS = None
other_cols_to_keep = ['timezoneoffset', 'aep_derived_uom']
mix_agg_functions=True
freqs=['H', '4H','D']
ugh_df_H_agg_newest = AMINonVee.build_time_resampled_aggregate_of_meters_df(df_15T, time_col_for_meter_agg=time_col_for_meter_agg, 
                                                                  agg_cols=agg_cols, agg_types=agg_types, 
                                                                  agg_dict_meter_agg=agg_dict_meter_agg, agg_dict_TRS=agg_dict_TRS, 
                                                                  other_cols_to_keep=other_cols_to_keep, freq=freqs, mix_agg_functions=mix_agg_functions)
print(ugh_df_H_agg_old.equals(ugh_df_H_agg_newest))

False


In [80]:
ugh_df_H_agg_newest.keys()

dict_keys(['15T', 'H', '4H', 'D'])

In [81]:
ugh_df_H_agg_newest['H']

,mean_TRS mean_mtrs value,sum_TRS mean_mtrs value,count_TRS mean_mtrs value,mean_TRS sum_mtrs value,sum_TRS sum_mtrs value,count_TRS sum_mtrs value,mean_TRS count_mtrs value,sum_TRS count_mtrs value,count_TRS count_mtrs value,timezoneoffset,aep_derived_uom
time_idx,,,,,,,,,,,
2018-10-04 16:00:00,0.138200,0.276400,2,1.3820,2.764,2,10.0,20,2,-1 days +20:00:00,KWH
2018-10-04 17:00:00,0.118300,0.473200,4,1.1830,4.732,4,10.0,40,4,-1 days +20:00:00,KWH
2018-10-04 18:00:00,0.146650,0.586600,4,1.4665,5.866,4,10.0,40,4,-1 days +20:00:00,KWH
2018-10-04 19:00:00,0.211700,0.846800,4,2.1170,8.468,4,10.0,40,4,-1 days +20:00:00,KWH
2018-10-04 20:00:00,0.164000,0.656000,4,1.6400,6.560,4,10.0,40,4,-1 days +20:00:00,KWH
...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01 00:00:00,0.098375,0.393500,4,1.1805,4.722,4,12.0,48,4,-1 days +20:00:00,KWH
2021-10-01 01:00:00,0.080417,0.321667,4,0.9650,3.860,4,12.0,48,4,-1 days +20:00:00,KWH
2021-10-01 02:00:00,0.057167,0.228667,4,0.6860,2.744,4,12.0,48,4,-1 days +20:00:00,KWH


In [82]:
df_H_agg_new = AMINonVee.build_time_resampled_df(df_15T_agg_new, freq=freqs, other_grouper_cols=[], agg_cols=['mean_mtrs value', 'sum_mtrs value', 'count_mtrs value'], agg_types=agg_types)

In [83]:
df_H_agg_new.keys()

dict_keys(['15T', 'H', '4H', 'D'])

In [84]:
df_15T

,serialnumber,prem_nb,srvc_pole_nb,trsf_pole_nb,annual_kwh,station_nb,xfmr_nb,starttimeperiod,endtimeperiod,aep_endtime_utc,timezoneoffset,aep_derived_uom,aep_srvc_qlty_idntfr,value,aep_usage_dt,aep_endtime_utc_from_timestamp,starttimeperiod_utc,endtimeperiod_utc
time_idx,,,,,,,,,,,,,,,,,,
2018-10-04 16:30:00,880693921,108248681,1832845704610,1832852704749,6530.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,RECEIVED,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880693917,102048681,1832852704749,1832852704749,5851.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,RECEIVED,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880687438,109826791,1832852704749,1832852704749,6078.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,RECEIVED,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880693921,108248681,1832845704610,1832852704749,6530.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,TOTAL,0.000,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
2018-10-04 16:30:00,880687439,101436791,1832858704845,1832852704749,8022.0,27,NaN,2018-10-04T12:15:00-04:00,2018-10-04T12:30:00-04:00,1538670600,-1 days +20:00:00,KWH,TOTAL,0.584,2018-10-04,2018-10-04 16:30:00,2018-10-04 16:15:00,2018-10-04 16:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01 04:00:00,880693921,108248681,1832845704610,1832852704749,6530.0,27,NaN,2021-09-30T23:45:00-04:00,2021-10-01T00:00:00-04:00,1633060800,-1 days +20:00:00,KWH,TOTAL,0.080,2021-09-30,2021-10-01 04:00:00,2021-10-01 03:45:00,2021-10-01 04:00:00
2021-10-01 04:00:00,880713197,108236791,1832858704845,1832852704749,4356.0,27,NaN,2021-09-30T23:45:00-04:00,2021-10-01T00:00:00-04:00,1633060800,-1 days +20:00:00,KWH,RECEIVED,0.000,2021-09-30,2021-10-01 04:00:00,2021-10-01 03:45:00,2021-10-01 04:00:00
2021-10-01 04:00:00,880687438,109826791,1832852704749,1832852704749,6078.0,27,NaN,2021-09-30T23:45:00-04:00,2021-10-01T00:00:00-04:00,1633060800,-1 days +20:00:00,KWH,TOTAL,0.050,2021-09-30,2021-10-01 04:00:00,2021-10-01 03:45:00,2021-10-01 04:00:00


In [85]:
df_15T = df_15T_OG.copy()

freqs=['H', '4H', 'D', 'MS']
#freqs=['H']
time_col_for_agg='endtimeperiod_utc'
other_grouper_cols=['serialnumber']
build_agg_dfs=True
other_cols_to_keep = ['prem_nb', 'srvc_pole_nb', 'trsf_pole_nb', 'station_nb', 
                      'xfmr_nb', 'aep_derived_uom', 'aep_srvc_qlty_idntfr']
other_cols_to_keep_agg = ['aep_derived_uom', 'aep_srvc_qlty_idntfr']
agg_cols=['value', 'annual_kwh']
agg_types=['mean']
mix_agg_functions=False

#-------------------------------------------------------------------------------------------------
resampled_dfs_kwh_dict = get_resampled_dfs(df_15T, base_freq='15T', freqs=freqs, 
                                           other_grouper_cols=other_grouper_cols, other_cols_to_keep=other_cols_to_keep, 
                                           build_agg_dfs=build_agg_dfs, time_col_for_agg=time_col_for_agg, 
                                           agg_cols=agg_cols, agg_types=agg_types, 
                                           other_cols_to_keep_agg=other_cols_to_keep_agg, mix_agg_functions=mix_agg_functions, 
                                           df_key=df_key, df_agg_key=df_agg_key)

In [86]:
resampled_dfs_kwh_dict.keys()

dict_keys(['15T', 'H', '4H', 'D', 'MS'])

In [87]:
# resampled_dfs_kwh_dictNEWest = AMINonVee.build_time_resampled_dfs(df_15T, base_freq='15T', freqs=freqs, 
#                                      other_grouper_cols=other_grouper_cols, other_cols_to_keep=other_cols_to_keep, 
#                                      build_agg_dfs=build_agg_dfs, time_col_for_agg=time_col_for_agg, 
#                                      agg_cols=agg_cols, agg_types=agg_types, 
#                                      other_cols_to_keep_agg=other_cols_to_keep_agg, mix_agg_functions=mix_agg_functions, 
#                                      df_key=df_key, df_agg_key=df_agg_key)

In [88]:
# assert(resampled_dfs_kwh_dictNEWest.keys()==resampled_dfs_kwh_dictNEW.keys())
# for time_key in resampled_dfs_kwh_dictNEWest.keys():
#     assert(resampled_dfs_kwh_dictNEWest[time_key].keys()==resampled_dfs_kwh_dictNEW[time_key].keys())
#     for df_type_key in resampled_dfs_kwh_dictNEWest[time_key].keys():
#         print(time_key)
#         print(df_type_key)
#         print(f"dfs equal?: {resampled_dfs_kwh_dictNEWest[time_key][df_type_key].equals(resampled_dfs_kwh_dictNEW[time_key][df_type_key])}")
#         print()

In [89]:
resampled_dfs_kwh_dictNEW = AMINonVee.build_time_resampled_dfs(df_15T, base_freq='15T', freqs=freqs, 
                                     other_grouper_cols=other_grouper_cols, other_cols_to_keep=other_cols_to_keep, 
                                     build_agg_dfs=build_agg_dfs, time_col_for_agg=time_col_for_agg, 
                                     agg_cols=agg_cols, agg_types=agg_types, 
                                     other_cols_to_keep_agg=other_cols_to_keep_agg, mix_agg_functions=mix_agg_functions, 
                                     df_key=df_key, df_agg_key=df_agg_key)

In [90]:
resampled_dfs_kwh_dictNEW['H'][df_key]

,serialnumber,mean_TRS value,mean_TRS annual_kwh,prem_nb,srvc_pole_nb,trsf_pole_nb,station_nb,xfmr_nb,aep_derived_uom,aep_srvc_qlty_idntfr,date
time_idx,,,,,,,,,,,
2018-10-04 16:00:00,880687438,0.804,6078.0,109826791,1832852704749,1832852704749,27,NaN,KWH,TOTAL,2018-10-04 16:00:00
2018-10-04 16:00:00,880687439,0.578,8022.0,101436791,1832858704845,1832852704749,27,NaN,KWH,TOTAL,2018-10-04 16:00:00
2018-10-04 16:00:00,880693914,0.000,9043.0,105148681,1832845704610,1832852704749,27,NaN,KWH,TOTAL,2018-10-04 16:00:00
2018-10-04 16:00:00,880693917,0.000,5851.0,102048681,1832852704749,1832852704749,27,NaN,KWH,TOTAL,2018-10-04 16:00:00
2018-10-04 16:00:00,880693921,0.000,6530.0,108248681,1832845704610,1832852704749,27,NaN,KWH,TOTAL,2018-10-04 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01 04:00:00,880687439,0.194,8022.0,101436791,1832858704845,1832852704749,27,NaN,KWH,TOTAL,2021-10-01 04:00:00
2021-10-01 04:00:00,880693914,0.174,9043.0,105148681,1832845704610,1832852704749,27,NaN,KWH,TOTAL,2021-10-01 04:00:00
2021-10-01 04:00:00,880693917,0.052,5851.0,102048681,1832852704749,1832852704749,27,NaN,KWH,TOTAL,2021-10-01 04:00:00


In [91]:
assert(resampled_dfs_kwh_dict.keys()==resampled_dfs_kwh_dictNEW.keys())
for time_key in resampled_dfs_kwh_dict.keys():
    assert(resampled_dfs_kwh_dict[time_key].keys()==resampled_dfs_kwh_dictNEW[time_key].keys())
    for df_type_key in resampled_dfs_kwh_dict[time_key].keys():
        print(time_key)
        print(df_type_key)
        print(f"dfs equal?: {resampled_dfs_kwh_dict[time_key][df_type_key].equals(resampled_dfs_kwh_dictNEW[time_key][df_type_key])}")
        print()

15T
df
dfs equal?: True

15T
df_agg
dfs equal?: False

H
df
dfs equal?: True

H
df_agg
dfs equal?: True

4H
df
dfs equal?: True

4H
df_agg
dfs equal?: True

D
df
dfs equal?: True

D
df_agg
dfs equal?: True

MS
df
dfs equal?: True

MS
df_agg
dfs equal?: True



In [ ]:
# TODO old_df_agg and new_df_agg were different before, as one had NaN values whereas in other these indices simply missing
# old_df_agg.shape = 
# new_df_agg.shape = 
# MAY BE EQUAL NOW.  this would be due to change in build_time_resampled_aggregate_of_meters_df where now freq=base_freq is
# not run through the uneccesary procedure, since it isn't being aggregated further

In [92]:
old_df = resampled_dfs_kwh_dict['15T']['df']
old_df_agg = resampled_dfs_kwh_dict['15T']['df_agg']

new_df = resampled_dfs_kwh_dictNEW['15T']['df']
new_df_agg = resampled_dfs_kwh_dictNEW['15T']['df_agg']

In [93]:
print(f'old_df.shape = {old_df.shape}')
print(f'new_df.shape = {new_df.shape}')

old_df.shape = (614097, 19)
new_df.shape = (614097, 19)


In [94]:
print(f'old_df_agg.shape = {old_df_agg.shape}')
print(f'new_df_agg.shape = {new_df_agg.shape}')

old_df_agg.shape = (104059, 5)
new_df_agg.shape = (104879, 5)


In [95]:
# df_15T.head()

In [96]:
old_df_agg.head()

,mean_mtrs value,mean_mtrs annual_kwh,aep_derived_uom,aep_srvc_qlty_idntfr,endtimeperiod_utc
time_idx,,,,,
2018-10-04 16:30:00,0.2788,7104.8,KWH,TOTAL,2018-10-04 16:30:00
2018-10-04 16:45:00,0.2740,7104.8,KWH,TOTAL,2018-10-04 16:45:00
2018-10-04 17:00:00,0.2692,7104.8,KWH,TOTAL,2018-10-04 17:00:00
2018-10-04 17:15:00,0.2724,7104.8,KWH,TOTAL,2018-10-04 17:15:00
2018-10-04 17:30:00,0.2212,7104.8,KWH,TOTAL,2018-10-04 17:30:00


In [97]:
new_df_agg.head()

,mean_TRS mean_mtrs value,mean_TRS mean_mtrs annual_kwh,aep_derived_uom,aep_srvc_qlty_idntfr,endtimeperiod_utc
time_idx,,,,,
2018-10-04 16:30:00,0.2788,7104.8,KWH,TOTAL,2018-10-04 16:30:00
2018-10-04 16:45:00,0.2740,7104.8,KWH,TOTAL,2018-10-04 16:45:00
2018-10-04 17:00:00,0.2692,7104.8,KWH,TOTAL,2018-10-04 17:00:00
2018-10-04 17:15:00,0.2724,7104.8,KWH,TOTAL,2018-10-04 17:15:00
2018-10-04 17:30:00,0.2212,7104.8,KWH,TOTAL,2018-10-04 17:30:00


In [98]:
ugh_idxs = [x for x in new_df_agg.index if x not in old_df_agg.index]
print(f'len(ugh_idxs) = {len(ugh_idxs)}')

len(ugh_idxs) = 820


In [99]:
new_df_agg.loc[ugh_idxs]

,mean_TRS mean_mtrs value,mean_TRS mean_mtrs annual_kwh,aep_derived_uom,aep_srvc_qlty_idntfr,endtimeperiod_utc
time_idx,,,,,
2019-01-02 05:15:00,NaN,NaN,None,None,2019-01-02 05:15:00
2019-01-02 05:30:00,NaN,NaN,None,None,2019-01-02 05:30:00
2019-01-02 05:45:00,NaN,NaN,None,None,2019-01-02 05:45:00
2019-01-02 06:00:00,NaN,NaN,None,None,2019-01-02 06:00:00
2019-01-02 06:15:00,NaN,NaN,None,None,2019-01-02 06:15:00
...,...,...,...,...,...
2021-02-25 21:45:00,NaN,NaN,None,None,2021-02-25 21:45:00
2021-02-25 22:00:00,NaN,NaN,None,None,2021-02-25 22:00:00
2021-02-25 22:15:00,NaN,NaN,None,None,2021-02-25 22:15:00


In [100]:
# new_df_agg and old_df_agg are different in the same way as all that bullshit above
# i.e. missing times incoluded as NaN vs not included!

In [101]:
resampled_dfs_kwh_dictNEW_unflat = AMINonVee.build_time_resampled_dfs(df_15T, base_freq='15T', freqs=freqs, 
                                     other_grouper_cols=other_grouper_cols, other_cols_to_keep=other_cols_to_keep, 
                                     build_agg_dfs=build_agg_dfs, time_col_for_agg=time_col_for_agg, 
                                     agg_cols=agg_cols, agg_types=agg_types, 
                                     other_cols_to_keep_agg=other_cols_to_keep_agg, mix_agg_functions=mix_agg_functions, 
                                     df_key=df_key, df_agg_key=df_agg_key, flatten_index=False)

In [102]:
assert(resampled_dfs_kwh_dictNEW_unflat.keys()==resampled_dfs_kwh_dictNEW.keys())
for time_key in resampled_dfs_kwh_dictNEW_unflat.keys():
    assert(resampled_dfs_kwh_dictNEW_unflat[time_key].keys()==resampled_dfs_kwh_dictNEW[time_key].keys())
    for df_type_key in resampled_dfs_kwh_dictNEW_unflat[time_key].keys():
        print(time_key)
        print(df_type_key)
        print(f"dfs equal?: {resampled_dfs_kwh_dictNEW_unflat[time_key][df_type_key].equals(resampled_dfs_kwh_dictNEW[time_key][df_type_key])}")
        print()

15T
df
dfs equal?: False

15T
df_agg
dfs equal?: True

H
df
dfs equal?: False

H
df_agg
dfs equal?: True

4H
df
dfs equal?: False

4H
df_agg
dfs equal?: True

D
df
dfs equal?: False

D
df_agg
dfs equal?: True

MS
df
dfs equal?: False

MS
df_agg
dfs equal?: True



In [113]:
tmp_df1 = resampled_dfs_kwh_dictNEW_unflat['H']['df']
tmp_df1.head()

mean_TRS value  mean_TRS annual_kwh  \
time_idx            serialnumber                                        
2018-10-04 16:00:00 880687438              0.804               6078.0   
                    880687439              0.578               8022.0   
                    880693914              0.000               9043.0   
                    880693917              0.000               5851.0   
                    880693921              0.000               6530.0   

                                    prem_nb   srvc_pole_nb   trsf_pole_nb  \
time_idx            serialnumber                                            
2018-10-04 16:00:00 880687438     109826791  1832852704749  1832852704749   
                    880687439     101436791  1832858704845  1832852704749   
                    880693914     105148681  1832845704610  1832852704749   
                    880693917     102048681  1832852704749  1832852704749   
                    880693921     108248681  1832845704610  1832852704749   

                                  station_nb  xfmr_nb aep_derived_uom  \
time_idx            serialnumber                                        
2018-10-04 16:00:00 880687438             27      NaN             KWH   
                    880687439             27      NaN             KWH   
                    880693914             27      NaN             KWH   
                    880693917             27      NaN             KWH   
                    880693921             27      NaN             KWH   

                                 aep_srvc_qlty_idntfr                date  
time_idx            serialnumber                                           
2018-10-04 16:00:00 880687438                   TOTAL 2018-10-04 16:00:00  
                    880687439                   TOTAL 2018-10-04 16:00:00  
                    880693914                   TOTAL 2018-10-04 16:00:00  
                    880693917                   TOTAL 2018-10-04 16:00:00  
                    880693921                   TOTAL 2018-10-04 16:00:00

In [114]:
tmp_df2 = resampled_dfs_kwh_dictNEW['H']['df']
tmp_df2.head()

,serialnumber,mean_TRS value,mean_TRS annual_kwh,prem_nb,srvc_pole_nb,trsf_pole_nb,station_nb,xfmr_nb,aep_derived_uom,aep_srvc_qlty_idntfr,date
time_idx,,,,,,,,,,,
2018-10-04 16:00:00,880687438,0.804,6078.0,109826791,1832852704749,1832852704749,27,NaN,KWH,TOTAL,2018-10-04 16:00:00
2018-10-04 16:00:00,880687439,0.578,8022.0,101436791,1832858704845,1832852704749,27,NaN,KWH,TOTAL,2018-10-04 16:00:00
2018-10-04 16:00:00,880693914,0.000,9043.0,105148681,1832845704610,1832852704749,27,NaN,KWH,TOTAL,2018-10-04 16:00:00
2018-10-04 16:00:00,880693917,0.000,5851.0,102048681,1832852704749,1832852704749,27,NaN,KWH,TOTAL,2018-10-04 16:00:00
2018-10-04 16:00:00,880693921,0.000,6530.0,108248681,1832845704610,1832852704749,27,NaN,KWH,TOTAL,2018-10-04 16:00:00


In [112]:
print(f'tmp_df1.shape = {tmp_df1.shape}')
print(f'tmp_df2.shape = {tmp_df2.shape}')

tmp_df1.shape = (104879, 5)
tmp_df2.shape = (104879, 5)


In [106]:
assert(tmp_df1.shape==tmp_df2.shape)

AssertionError: 

In [ ]:
tmp_df1.index.get_level_values(0)

In [ ]:
tmp_df2.index.get_level_values(0)

In [ ]:
all(tmp_df1.index.get_level_values(0)==tmp_df2.index.get_level_values(0))

In [ ]:
tmp_df1.equals(tmp_df2.rename(columns=dict(zip(tmp_df2.columns, tmp_df1.columns))))

In [ ]:
tmp_df1b = tmp_df1.copy()
tmp_df1b.columns = tmp_df1b.columns.to_flat_index()
tmp_df1b = tmp_df1b.rename(columns=dict(zip(tmp_df1.columns, tmp_df2.columns)))
tmp_df1b.head()

In [ ]:
tmp_df1b.equals(tmp_df2)

# TODO!!!!!!!!!!!!!!!!!!!!!!!!!!!!
maybe don't want freqs to be altered via freq.insert(0,base_freq)?
More specifically, freqs is changed outside of the function 

In [ ]:
df_H_we = AMINonVee.build_time_resampled_df(df_15T_agg_new, 'H', other_grouper_cols=['timezoneoffset'], other_cols_to_keep=['aep_derived_uom'], agg_cols=['mean_mtrs value', 'sum_mtrs value', 'count_mtrs value'], agg_types=agg_types)

In [ ]:
df_H_we.index.nlevels